In [3]:
import time
from coffea import hist, nanoevents, util
import coffea.processor as processor
import awkward as ak
import numpy as np
import glob as glob
import re
import itertools
# import vector as vec
from coffea.nanoevents.methods import vector, candidate
from coffea.nanoevents import NanoAODSchema, BaseSchema
from coffea.lumi_tools import LumiMask
# for applying JECs
from coffea.lookup_tools import extractor
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
from coffea.jetmet_tools import JetResolution, JetResolutionScaleFactor
# from jmeCorrections import JetMetCorrections
import json


import coffea.processor as processor
from coffea import hist
from coffea.analysis_tools import PackedSelection, Weights
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory
from coffea.nanoevents.methods import nanoaod

# NanoAODSchema.warn_missing_crossrefs = False

import pickle
# np.errstate(invalid='ignore', divide='ignore')

In [4]:
fileset = {
    'MET': 
    ["root://xrootd-cms.infn.it//store/user/swieland/customNano/MET/KITv2_CustomNanoV9_Data_2018B/220830_111532/0000/DATA_2018_NanoAODv9_10.root"],
    'WJetsToLNu_Pt-400To600_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8____1_':
[
            "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/0BD074B6-C3F6-B144-A7CE-30B822187BF8.root",
#             "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/9AC5AECD-9063-D040-AA99-09F9308A0238.root",
#             "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/E22B3E7A-1944-1744-93DE-EF53D0635E92.root"
        ],
    'sig': ["root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_1.root"]
}
#fname = fileset["WJetsToLNu_Pt-400To600_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8____1_"][0]
fname = fileset["MET"][0]
#fname = fileset['sig']
events = NanoEventsFactory.from_root(fname, schemaclass=NanoAODSchema).events()

OSError: XRootD error: [ERROR] Operation expired
in file root://xrootd-cms.infn.it//store/user/swieland/customNano/MET/KITv2_CustomNanoV9_Data_2018B/220830_111532/0000/DATA_2018_NanoAODv9_10.root

In [2]:
fnameMC = "./Mphi-1995_Mchi-1000_2018_1.root"
fname="Mphi-1995_Mchi-1000_2018"

In [8]:
class Newttag(processor.ProcessorABC):
    def __init__(self):
        self._fields = """
        AK15PFPuppi_Jet_pt
        AK15PFPuppi_Jet_phi
        AK15PFPuppi_Jet_eta
        AK15PFPuppi_Jet_particleNetAK15_TvsQCD
        GenPart_eta
        GenPart_phi
        GenPart_pt
        GenPart_pdgId
        GenPart_status
        GenPart_statusFlags
        nGenPart
        FatJet_phi
        FatJet_pt
        FatJet_eta
        FatJet_mass
        nFatJet
        """.split()
        
        self._accumulator = processor.dict_accumulator({
            "ak15jetpt": hist.Hist(
                "Events",
                hist.Cat("dataset", "Dataset"),
                hist.Bin("ak15jetpt", "AK15 Jet pT", 100, 0, 2000),
            )
        })
    @property
    def accumulator(self):
        return self._accumulator
    
    @property
    def fields(self):
        return self._fields
    
    def process(self, events):
        dataset = events.metadata['dataset']
        selection = processor.PackedSelection()
        output = self.accumulator.identity()
        
        
        mu = events.Muon
        leading_mu = mu[:,:1]
        mu = mu[ak.num(mu) >= 2]
        print('type mu: ', type(mu)) 
#        print('leading: ', leading_mu)
        print('2nd? ', mu[:,:1])
        def find_2lep(events_leptons, builder):
            for leptons in events_leptons:
                builder.begin_list()
                nlep = len(leptons)
                for i0 in range(nlep):
                    for i1 in range(i0 + 1, nlep):
                        if leptons[i0].charge + leptons[i1].charge != 0:
                            continue

                        if len({i0, i1}) < 2:
                            continue

                        builder.begin_tuple(4)
                        builder.index(0).integer(i0)
                        builder.index(1).integer(i1)
                        builder.end_tuple()
                builder.end_list()

            return builder
        
        mu_pair = find_2lep(mu, ak.ArrayBuilder()).snapshot()
        mu_pair = [mu[mu_pair[idx]] for idx in "01"]
        mu_Pair = ak.zip({
            'i0': mu[:,:1],
            'i1': mu[:,1:2],
            'dimu': mu[:,:1] + mu[:,1:2]
        })
        
        print('mu_pair0: ', mu_pair[0].pt)
        #print('mu_pair0: ', mu_pair[0].fields)
        print('mu_pair1: ', mu_pair[1].pt)
        #print('mu_pair1: ', mu_pair[1].fields)
        #print('mu_pair pt: ', mu_pair.pt)
        dimu = mu_pair[0] + mu_pair[1]
        #dimu2 = mu[:,:1] + mu[:,1:2]
        
        #print('dimu:', dimu.pt)
        print('mu_pair:', mu_pair)
        #print('dimu2.pt:', dimu2.pt)
        #print('mu_Pair.pt', mu_Pair.pt) 
        print('i0:', mu_Pair.i0.pt)
        print('i1:', mu_Pair.i1.pt)
        print('di:', mu_Pair.dimu.pt)

        
        e = events.Electron
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [9]:
fnameMC = "./Mphi-1995_Mchi-1000_2018_1.root"
fnameMCs = [
    "root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_1.root",
    "root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_2.root",
    "root://cmseos.fnal.gov//store/group/lpcmetx/Monotop/CustomNanoProdV9/2018/Mphi-1995_Mchi-1000_2018/Mphi-1995_Mchi-1000_2018_3.root"
]
onefiles = [
    "./Mphi-1995_Mchi-1000_2018_1.root",
    "root://xrootd-cms.infn.it//store/user/empfeffe/customNano/DYJetsToLL_0J_TuneCP5_13TeV-amcatnloFXFX-pythia8/KIT_CustomNanoV9_MC_2018/211220_132108/0000/MC_2018_NanoAODv9_189.root",
]
samples = {
    "signal": fnameMCs
    #"signal": [fnameMC]
    #"dy": onefiles
}

In [10]:
result = processor.run_uproot_job(
    samples,
    "Events",
    Newttag(),
    #processor.iterative_executor,
    processor.futures_executor,
    #{"schema": NanoAODSchema},
    executor_args={'schema': NanoAODSchema,'workers': 8},
)



Processing:   0%|          | 0/5 [00:00<?, ?chunk/s]

type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
mu_pair0:  [[39.9, 11.1], [], [], [], [15.6], [], ... [], [], [], [15.6], [46.6, 23.2], []]
mu_pair1:  [[11.1, 8.86], [], [], [], [3.01], [], ... [], [], [], [3.32], [3.62, 3.62], []]
mu_pair: [<MuonArray [[Muon, Muon], [], .

In [15]:
result = processor.run_uproot_job(
    samples,
    "Events",
    Newttag(),
    #processor.iterative_executor,
    processor.futures_executor,
    #{"schema": NanoAODSchema},
    executor_args={'schema': NanoAODSchema,'workers': 8},
)



Processing:   0%|          | 0/5 [00:00<?, ?chunk/s]

type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>2nd? 
 2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]][[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]

type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
type mu:  <class 'coffea.nanoevents.methods.nanoaod.MuonArray'>
2nd?  [[Muon], [Muon], [Muon], [Muon], [Muon], ... [Muon], [Muon], [Muon], [Muon], [Muon]]
mu_pair0:  [[39.9, 11.1], [], [], [], [15.6], [], ... [], [], [], [15.6], [46.6, 23.2], []]
mu_pair1:  [[11.1, 8.86], [], [], [], [3.01], [], ... [], [], [], [3.32], [3.62, 3.62], []]
dimu: [[50.4, 19.9], [], [], [], [18], []

In [8]:
class DiJet(processor.ProcessorABC):
    def __init__(self):
        
        self._accumulator = processor.dict_accumulator({
            "ak15jetpt": hist.Hist(
                "Events",
                hist.Cat("dataset", "Dataset"),
                hist.Bin("ak15jetpt", "AK15 Jet pT", 100, 0, 2000),
            )
        })
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        dataset = events.metadata['dataset']
        output = self.accumulator.identity()
                
        mu = events.Muon
        leading_mu = mu[:,:1]
        mu = mu[ak.num(mu) >= 2]
        print('type mu: ', type(mu)) 
#        print('leading: ', leading_mu)
        print('2nd? ', mu[:,:1])
        def find_2lep(events_leptons, builder):
            for leptons in events_leptons:
                builder.begin_list()
                nlep = len(leptons)
                for i0 in range(nlep):
                    for i1 in range(i0 + 1, nlep):
                        if leptons[i0].charge + leptons[i1].charge != 0:
                            continue

                        if len({i0, i1}) < 2:
                            continue

                        builder.begin_tuple(4)
                        builder.index(0).integer(i0)
                        builder.index(1).integer(i1)
                        builder.end_tuple()
                builder.end_list()

            return builder
        
        mu_pair = find_2lep(mu, ak.ArrayBuilder()).snapshot()
        mu_pair = [mu[mu_pair[idx]] for idx in "01"]
        mu_Pair = ak.zip({
            'i0': mu[:,:1],
            'i1': mu[:,1:2],
            'dimu': mu[:,:1] + mu[:,1:2]
        })
        
        print('mu_pair0: ', mu_pair[0].pt)
        j = = events.Jet
        j_w_two_jets = j[ak.num(j) >= 2]
        dijet = j_w_two_jets[:,0] + j_w_two_jets[:,1]


        
        e = events.Electron
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [ ]:
result = processor.run_uproot_job(
    samples,
    "Events",
    DiJet(),
    #processor.iterative_executor,
    processor.futures_executor,
    #{"schema": NanoAODSchema},
    executor_args={'schema': NanoAODSchema,'workers': 8},
)
